In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta as rd
import time
import math
from sklearn.metrics import mean_squared_error

In [2]:
# stocks data csv read
df = pd.read_csv('/Users/apple/Downloads/data.csv')
df = df.set_index('Date')

# s&p data csv read
df_sp = pd.read_csv('/Users/apple/Downloads/sp500.csv')
df_sp = df_sp.set_index('Date')

# stocks data csv read for partial replication
df_reduce = pd.read_csv('/Users/apple/Downloads/data.csv')
df_reduce = df_reduce.set_index('Date')

In [5]:
df.shape

(1385, 471)

In [6]:
df_sp.shape

(1385, 1)

In [7]:
df_reduce.shape

(1385, 471)

In [8]:
def date_slicer(df, start, duration, rebalancing_period=0):
    '''
    this function is used to slice out specific section of the data
    '''
    start = str(datetime.strptime(start, '%Y-%m-%d').date() + rd(months=rebalancing_period))
    end = str(datetime.strptime(start, '%Y-%m-%d').date() + rd(months=duration) - rd(days=1))
    return df.loc[start:end]

In [9]:
def data_process(df):
    '''
    this function gets the dataframe as input, processes it, and ouputs the cumulative change of the stocks
    that is used as input for training the model.
    '''
    df = df.pct_change()
    df = df.tail(-1)
    df = df + 1
    df = df.cumprod()
    df = df - 1
    df = df.iloc[-1,:]
    df = df.to_numpy()
    df = torch.from_numpy(df).type(torch.Tensor)
    return df

In [10]:
def daily_change(df):
    '''
    this function calculate the daily change of stocks included in the dataframe.
    '''
    df = df.pct_change()
    df = df.tail(-1)
    return df

In [11]:
def daily_return(df):
    '''
    this function calculate the daily return of stocks included in the dataframe, note that 
    daily return is equal to daily change + 1
    '''
    df = df.pct_change()
    df = df.tail(-1)
    df = df + 1
    return df

In [12]:
def index_finder(df):
    '''
    this function is just being used for extracting the stocks symbols
    '''
    df = df.pct_change()
    df = df.tail(-1)
    df = df + 1
    df = df.cumprod()
    df = df - 1
    df = df.iloc[-1,:]
    return df

In [13]:
# storing stocks symbols
stocks_index = index_finder(df).index

In [14]:
# shallow nnf biuld
class shallow_NNF(nn.Module):
    '''
    this class is used to train the data with Shallow NNF model, consisted of 2 fully connected layers, 
    a relU activation function in between and a softmax layer output that is translated into stock weights in portfolio.
    '''
    def __init__(self, input_dim, hidden_size, num_classes):
        super(shallow_NNF, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size) # fully connected layer
        self.fc2 = nn.Linear(hidden_size, num_classes) # fully connected layer
        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=0)
        
    def reset_parameters(self):
        self.fc1.reset_parameters()
        self.fc2.reset_parameters()
        
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.softmax(self.fc2(out))
        weights = out
        cumulative_change = sum(out * x)
        return cumulative_change, weights

In [15]:
'''
shallow nnf partial biuld which is the same as original shallow nnf
this class helps us to use the full replication training to find the best companies to invest
and then find the optimal wieghts with the partial model
'''
class shallow_NNF_partial(nn.Module):
    def __init__(self, input_dim, hidden_size, num_classes):
        super(shallow_NNF_partial, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=0)
        
    def reset_parameters(self):
        self.fc1.reset_parameters()
        self.fc2.reset_parameters()
        
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.softmax(self.fc2(out))
        weights = out
        cumulative_change = sum(out * x)
        return cumulative_change, weights

In [19]:
# rebalancing period = one or three months
rbp = 1

# number of companies in the partial portfolio
partial_num = 50

# epochs
num_epochs = 100

In [20]:
# shallow_nnf hyperparameters
input_dim = 471
hidden_size = 471
num_classes = 471
lr = 1e-3 # learning rate

In [21]:
# shallow nnf tune
'''
loss function is set to MSE and Adam optimizer is used in this model.
'''
shallow_NNF = shallow_NNF(input_dim=input_dim, hidden_size=hidden_size, num_classes=num_classes)
shallow_NNF_loss_fun = torch.nn.MSELoss(reduction='mean')
shallow_NNF_optimizer = torch.optim.Adam(shallow_NNF.parameters(), lr=lr)

In [22]:
# shallow nnf partial tune
'''
loss function is set to MSE and Adam optimizer is used in this model.
'''
shallow_NNF_partial = shallow_NNF_partial(input_dim=partial_num, hidden_size=hidden_size, num_classes=partial_num)
shallow_NNF_partial_loss_fun = torch.nn.MSELoss(reduction='mean')
shallow_NNF_partial_optimizer = torch.optim.Adam(shallow_NNF_partial.parameters(), lr=lr)

In [26]:
# RMSE
def RMSE(x, y, weights):
    '''
    this function calculates the root mean squere error of constructed portfollio and benchmark index 
    that is used for evaluating trained models.
    '''
    temp = 0
    for i in range(len(x)):
        temp += (sum(x.iloc[i] * weights) - y.iloc[i]) ** 2
    return math.sqrt(temp/len(x))

In [27]:
# MEAN
def MEAN(x, weights):
    '''
    this function calculates the mean return of the constructed portfolio during the given period.
    '''
    temp = []
    for i in range(len(x)):
        temp.append(sum(x.iloc[i] * weights))
    temp = np.array(temp)
    return temp.mean()

In [28]:
# Volatility
def VOL(x, weights):
    '''
    this function calculates the volatility of the constructed portfolio during the given period.
    '''
    temp = []
    for i in range(len(x)):
        temp.append(sum(x.iloc[i] * weights))
    temp = np.array(temp)
    return temp.std()

In [29]:
def portfolio_return(df, x_test, model, i, temp):   
    '''
    this function outputs the cumulative return of the portfolio test dataset of the given dataframe
    ''' 
    x_return = date_slicer(df, '2018-01-01', 1, i)
    x_return =  x_return.pct_change()
    x_return =  x_return.tail(-1)
    x_return =  x_return + 1
    x_return =  x_return.cumprod()
    
    if model == equal_w_model:
        weights = model(x_test).performance()[1]
    else:
        weights = np.array(model(x_test)[1].detach())
    
    for i in range(len(x_return)):
        temp.append(sum(x_return.iloc[i] * weights))
    temp = np.array(temp)
    return temp

In [30]:
def index_return(df_sp, i, temp):
    '''
    this function outputs the cumulative return of the benchmark index test dataset of the given dataframe
    '''
    y_return = date_slicer(df_sp, '2018-01-01', 1, i)
    y_return = y_return.pct_change()
    y_return = y_return.tail(-1)
    y_return = y_return + 1
    y_return = y_return.cumprod()
    
    for i in range(len(y_return)):
        temp.append(sum(y_return.iloc[i]))
    temp = np.array(temp)
    return temp

In [31]:
def valid_fun(x_valid, i, model):
    '''
    this function gets validation dataset, model and rebalaning period as input, then outputs the RMSE of given dataset.
    '''
    x_change = daily_change(date_slicer(df_reduce, '2017-07-01', 6, i))
    y_change = daily_change(date_slicer(df_sp, '2017-07-01', 6, i))
    # x_return = daily_return(date_slicer(df, '2017-07-01', 6, i))
    # y_return = daily_return(date_slicer(df_sp, '2017-07-01', 6, i))
    
    if model == equal_w_model:
        weights = model(x_valid).performance()[1]
    else:
        weights = np.array(model(x_valid)[1].detach())
    
    valid_rmse = RMSE(x_change, y_change, weights)
    # valid_mean = MEAN(x_return, weights)
    # valid_vol  = VOL(x_return, weights)
    
    print(f'Validation RMSE: {valid_rmse}')
    # print(f'Validation MEAN: {valid_mean}')
    # print(f'Validation VOL: {valid_vol}')
    
    return valid_rmse

In [32]:
def test_fun(x_test, i, model):
    '''
    this function gets test dataset, model and rebalaning period as input, then outputs the RMSE, Mean and volatility 
    of the given dataset.
    '''
    x_change = daily_change(date_slicer(df_reduce, '2018-01-01', 6, i))
    y_change = daily_change(date_slicer(df_sp, '2018-01-01', 6, i))
    x_return = daily_return(date_slicer(df_reduce, '2018-01-01', 6, i))
    y_return = daily_return(date_slicer(df_sp, '2018-01-01', 6, i))
    
    if model == equal_w_model:
        weights = model(x_test).performance()[1]
    else:
        weights = np.array(model(x_test)[1].detach())
    
    test_rmse = RMSE(x_change, y_change, weights)
    test_mean = MEAN(x_return, weights)
    test_vol  = VOL(x_return, weights)
    test_dic = {'RMSE': test_rmse, 'MEAN': test_mean, 'VOL': test_vol}
    
    print(f'Test RMSE: {test_rmse}')
    print(f'Test MEAN: {test_mean}')
    print(f'Test VOL: {test_vol}')
    
    return test_dic

In [36]:
# deep nnf build
class deep_NNF(nn.Module):
    '''
    this class is used to train the data with Deep NNF model, consisted of 6 fully connected layers, 
    relU activation functions in between and a softmax layer output that is translated into stock weights in portfolio.
    dropout is also included in deep NNF model.
    '''
    def __init__(self, input_dim, hidden_size1, hidden_size2, hidden_size3,
                 hidden_size4, hidden_size5, num_classes, dropout_p = 0.2):
        super(deep_NNF, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size1) # fully connected layer
        self.fc2 = nn.Linear(hidden_size1, hidden_size2) # fully connected layer
        self.fc3 = nn.Linear(hidden_size2, hidden_size3) # fully connected layer
        self.fc4 = nn.Linear(hidden_size3, hidden_size4) # fully connected layer
        self.fc5 = nn.Linear(hidden_size4, hidden_size5) # fully connected layer
        self.fc6 = nn.Linear(hidden_size5, num_classes) # fully connected layer
    
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_p)
        self.softmax = nn.Softmax(dim=0)
        
    def reset_parameters(self):
        self.fc1.reset_parameters()
        self.fc2.reset_parameters()
        self.fc3.reset_parameters()
        self.fc4.reset_parameters()
        self.fc5.reset_parameters()
        self.fc6.reset_parameters()
        
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.dropout(out)
        out = self.relu(self.fc2(out))
        out = self.dropout(out)
        out = self.relu(self.fc3(out))
        out = self.dropout(out)
        out = self.relu(self.fc4(out))
        out = self.dropout(out)
        out = self.relu(self.fc5(out))
        out = self.softmax(self.fc6(out))
        weights = out
        cumulative_change = sum(out * x)
        return cumulative_change, weights


In [39]:
# deep_nnf hyperparameters
input_dim = 471
hidden_size1 = 471
hidden_size2 = 471
hidden_size3 = 471
hidden_size4 = 471
hidden_size5 = 471
num_classes = 471
lr = 1e-7 # learning rate
# probability of a neuron being shutdown that shuffles every epoch minimizing the overfit phenomenon
dropout_p = 0.2

In [41]:
# deep nnf tune
'''
like in shallow NNF, loss function is set to MSE and Adam optimizer is used.
'''
deep_NNF = deep_NNF(input_dim=input_dim, hidden_size1=hidden_size1, hidden_size2=hidden_size2, 
                    hidden_size3=hidden_size3, hidden_size4=hidden_size4, hidden_size5=hidden_size5,
                    num_classes=num_classes)
deep_NNF_loss_fun = torch.nn.MSELoss(reduction='mean')
deep_NNF_optimizer = torch.optim.Adam(deep_NNF.parameters(), lr=lr)



In [43]:
def partial(x_train, x_valid, x_test, weights, stocks_index, num = partial_num):
    df_partial = pd.DataFrame({'x_train': x_train, 'x_valid': x_valid, 'x_test': x_test,
                               'weights': weights}, index = stocks_index)
    df_partial = df_partial.sort_values(by = ['weights'])
    out_index = df_partial.index[num:]
    df_partial = df_partial.iloc[:num]
    
    x_train = df_partial['x_train'].to_numpy()
    x_valid = df_partial['x_valid'].to_numpy()
    x_test = df_partial['x_test'].to_numpy()
    
    x_train = torch.from_numpy(x_train).type(torch.Tensor)
    x_valid = torch.from_numpy(x_valid).type(torch.Tensor)
    x_test = torch.from_numpy(x_test).type(torch.Tensor)
    
    return x_train, x_valid, x_test, out_index



### **Shallow NNF Training**

In [59]:
# shallow nnf training function
def train_shallow_nnf(x_train, y_train, i):
    '''
    this function is used to train the model using x_train & y_train given to it, printing MSE of trained model in first and last
    epoch and also printing train time of the model
    '''
    start_time_shallow_nnf = time.time()
    print(f'\nShallow NNF Training & Results for model {(i/rbp)+1}:')
    print("x_train.shape")
    print(x_train.shape)
    for epoch in range(num_epochs):
        y_train_pred = shallow_NNF(x_train)[0]
        loss_shallow_nnf = shallow_NNF_loss_fun(y_train_pred, y_train)
        if epoch == 0 or epoch == num_epochs-1:
            weights = np.array(deep_NNF(x_train)[1].detach())
            
            print("weights")
            print("deep_NNF(x_train)[1].shape")
            print(deep_NNF(x_train)[1].shape)

            print("deep_NNF(x_train)[0].shape")
            print(deep_NNF(x_train)[0].shape)
            
            print(f'Epoch {epoch+1} of {num_epochs} | MSE: {loss_shallow_nnf.item()}')
        shallow_NNF_optimizer.zero_grad()
        loss_shallow_nnf.backward()
        shallow_NNF_optimizer.step()
        
    training_time = format(time.time()-start_time_shallow_nnf, '0.2f')
    print(f'Training time: {training_time}')
    
    return weights

In [34]:
# shallow nnf partial training function
def train_shallow_nnf_partial(x_train, y_train, i):    
    start_time_shallow_nnf = time.time()
    print(f'\nDeep NNF Training & Results for model {(i/rbp)+1} (Partial replication):')
    
    for epoch in range(num_epochs):
        y_train_pred = shallow_NNF_partial(x_train)[0]
        loss_shallow_nnf = shallow_NNF_partial_loss_fun(y_train_pred, y_train)
        if epoch == 0 or epoch == num_epochs-1:
            print(f'Epoch {epoch+1} of {num_epochs} | MSE: {loss_shallow_nnf.item()}')
        shallow_NNF_partial_optimizer.zero_grad()
        loss_shallow_nnf.backward()
        shallow_NNF_partial_optimizer.step()
        
    training_time = format(time.time()-start_time_shallow_nnf, '0.2f')
    print(f'Training time: {training_time}')

In [66]:
#shallow nnf
'''
in this cell,firstly, train, validation and test datasets are sliced in each loop. then shallow NNf outputs the best stocks with 
full replication and then we use the specific stocks to train the model again and get the optimal weights (each loop)
then best model will be chosen. Also RMSE, Mean and volatility of all models and then the best model is printed.
'''
shallow_nnf_valid_rmse_list = []
shallow_nnf_test_results = []
shallow_nnf_test_plot = [] # storing the shallow model test data return for plotting later on

for i in range(int(24/rbp)):
    df_reduce = df.copy()
    
    x_train = data_process(date_slicer(df, '2014-07-01', 36, i*rbp))
    print("x_train.shape")
    print(x_train.shape)
    
    y_train = data_process(date_slicer(df_sp, '2014-07-01', 36, i*rbp))
    print("y_train.shape")
    print(y_train.shape)
    
    x_valid = data_process(date_slicer(df, '2017-07-01', 6, i*rbp))
    print("x_valid.shape")
    print(x_valid.shape)
    
    y_valid = data_process(date_slicer(df_sp, '2017-07-01', 6, i*rbp))
    print("y_valid.shape")
    print(y_valid.shape)
    
    x_test = data_process(date_slicer(df, '2018-01-01', 1, i*rbp))
    print("x_test.shape")
    print(x_test.shape)
    y_test = data_process(date_slicer(df_sp, '2018-01-01', 1, i*rbp))
    print("y_test.shape")
    print(y_test.shape)

    
    weights = train_shallow_nnf(x_train, y_train, i*rbp)
    
    print("weights.shape:")
    print(weights.shape)
    
    x_train, x_valid, x_test, out_index = partial(x_train, x_valid, x_test, weights, stocks_index, num = partial_num)
    
    df_reduce = df_reduce.drop(out_index, axis=1)
    train_shallow_nnf_partial(x_train, y_train, i*rbp)
    shallow_nnf_valid_rmse_list.append(valid_fun(x_valid, i*rbp, shallow_NNF_partial))
    shallow_nnf_test_results.append(test_fun(x_test, i*rbp, shallow_NNF_partial))
    portfolio_return(df_reduce, x_test, shallow_NNF_partial, i, shallow_nnf_test_plot)
    shallow_NNF.reset_parameters()
    shallow_NNF_partial.reset_parameters()

# print(f'\nMin Valid RMSE is: {min(valid_rmse_list)} for model i = {(deep_best_result_index)+1}')
print('Selected Model Test Results for model i =', (deep_best_result_index)+1, 'are: ')
print('RMSE =', shallow_nnf_test_results[(deep_best_result_index)]['RMSE'])
print('MEAN =', shallow_nnf_test_results[(deep_best_result_index)]['MEAN'])
print('VOL =', shallow_nnf_test_results[(deep_best_result_index)]['VOL'])

shallow_nnf_test_plot = np.array(shallow_nnf_test_plot).reshape(-1,1)

x_train.shape
torch.Size([471])
y_train.shape
torch.Size([1])
x_valid.shape
torch.Size([471])
y_valid.shape
torch.Size([1])
x_test.shape
torch.Size([471])
y_test.shape
torch.Size([1])

Shallow NNF Training & Results for model 1.0:
x_train.shape
torch.Size([471])
weights
deep_NNF(x_train)[1].shape
torch.Size([471])
deep_NNF(x_train)[0].shape
torch.Size([])
Epoch 1 of 100 | MSE: 0.04368942603468895
weights
deep_NNF(x_train)[1].shape
torch.Size([471])
deep_NNF(x_train)[0].shape
torch.Size([])
Epoch 100 of 100 | MSE: 1.7945667174501523e-10
Training time: 0.38
weights.shape:
(471,)

Deep NNF Training & Results for model 1.0 (Partial replication):
Epoch 1 of 100 | MSE: 0.11405888944864273
Epoch 100 of 100 | MSE: 9.18633134006086e-08
Training time: 0.07
Validation RMSE: 0.0021850764032716003
Test RMSE: 0.0020699087341954567
Test MEAN: 1.0002930009802125
Test VOL: 0.009512322622623656
x_train.shape
torch.Size([471])
y_train.shape
torch.Size([1])
x_valid.shape
torch.Size([471])
y_valid.shape
to

/var/folders/j9/ryqgqp6n03schy65y0b0g4sm0000gn/T/ipykernel_24518/1184317780.py:10: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return math.sqrt(temp/len(x))


weights
deep_NNF(x_train)[1].shape
torch.Size([471])
deep_NNF(x_train)[0].shape
torch.Size([])
Epoch 100 of 100 | MSE: 1.0231136826632792e-08
Training time: 0.40
weights.shape:
(471,)

Deep NNF Training & Results for model 2.0 (Partial replication):
Epoch 1 of 100 | MSE: 0.10646946728229523
Epoch 100 of 100 | MSE: 5.034713979057415e-08
Training time: 0.06
Validation RMSE: 0.002371994551996431
Test RMSE: 0.002482867513463893
Test MEAN: 1.0000678601854107
Test VOL: 0.009496709513081477
x_train.shape
torch.Size([471])
y_train.shape
torch.Size([1])
x_valid.shape
torch.Size([471])
y_valid.shape
torch.Size([1])
x_test.shape
torch.Size([471])
y_test.shape
torch.Size([1])

Shallow NNF Training & Results for model 3.0:
x_train.shape
torch.Size([471])
weights
deep_NNF(x_train)[1].shape
torch.Size([471])
deep_NNF(x_train)[0].shape
torch.Size([])
Epoch 1 of 100 | MSE: 0.0436181016266346
weights
deep_NNF(x_train)[1].shape
torch.Size([471])
deep_NNF(x_train)[0].shape
torch.Size([])
Epoch 100 of 100 

NameError: name 'deep_best_result_index' is not defined